In [4]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import json
import os

In [3]:
# data_dir = '/Users/jzhang14/python_notebooks/pydata-book/'
data_dir = 'E:\_Workspaces\JupyterNotebooks\pydata-book'

In [2]:
obj = """
{"name": "Wes",
"places_lived": ["United Status", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
                {"name": "Katie", "age": 33, "pet": "Cisco"}]

}
"""

In [4]:
result = json.loads(obj)

In [5]:
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United Status', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [6]:
asjson = json.dumps(result)

In [7]:
asjson

'{"name": "Wes", "places_lived": ["United Status", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Katie", "age": 33, "pet": "Cisco"}]}'

In [8]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])

In [9]:
siblings

,name,age
0,Scott,25
1,Katie,33


In [18]:
from lxml import html
import requests

In [19]:
page = requests.get('http://finance.yahoo.com/q/op?s=AAPL+Options')

In [33]:
tree = html.fromstring(page.text)

<Element html at 0xb3b85e8>

In [41]:
links = tree.findall('.//a')

In [42]:
links[15:20]

[<Element a at 0xb3b8f48>,
 <Element a at 0xb3b8f98>,
 <Element a at 0xb3b5048>,
 <Element a at 0xb3b5098>,
 <Element a at 0xb3b50e8>]

In [43]:
lnk = links[28]

In [44]:
lnk

<Element a at 0xb3b53b8>

In [45]:
lnk.get('href')

'/quote/NFLX?p=NFLX'

In [61]:
links[29].get('href')

'/quote/TSLA?p=TSLA'

In [47]:
lnk.text_content()

'NFLX'

In [48]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]

In [49]:
urls[-10:]

['https://smallbusiness.yahoo.com',
 'https://help.yahoo.com/kb/index?page=content&y=PROD_FIN_DESK&locale=en_US&id=SLN2310',
 'http://help.yahoo.com/l/us/yahoo/finance/',
 'https://yahoo.uservoice.com/forums/382977',
 'http://info.yahoo.com/privacy/us/yahoo/',
 'http://info.yahoo.com/relevantads/',
 'http://info.yahoo.com/legal/us/yahoo/utos/utos-173.html',
 'http://twitter.com/YahooFinance',
 'http://facebook.com/yahoofinance',
 'http://yahoofinance.tumblr.com']

In [62]:
tables = doc.findall('.//table')
tables

[<Element table at 0xb3bfe58>,
 <Element table at 0xb3bfcc8>,
 <Element table at 0xb3bff48>]

In [54]:
calls = tables[1]

In [55]:
puts = tables[2]

In [56]:
rows = calls.findall('.//tr')

In [57]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

In [58]:
_unpack(rows[0], kind='th')

['Strike',
 'Contract Name',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [59]:
_unpack(rows[1], kind='td')

['2.50',
 'AAPL170519C00002500',
 '153.15',
 '151.95',
 '154.40',
 '8.45',
 '5.84%',
 '4',
 '0',
 '2,237.50%']

In [63]:
from pandas.io.parsers import TextParser

In [65]:
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()
    

In [66]:
call_data = parse_options_data(calls)

In [67]:
put_data = parse_options_data(puts)

In [68]:
call_data[:10]

,Strike,Contract Name,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,2.5,AAPL170519C00002500,153.15,151.95,154.40,8.45,5.84%,4,0,"2,237.50%"
1,5.0,AAPL170519C00005000,150.65,149.45,151.65,0.00,0.00%,4,0,"1,562.50%"
2,7.5,AAPL170519C00007500,148.05,146.95,149.30,0.00,0.00%,3,0,"1,478.13%"
3,10.0,AAPL170519C00010000,143.53,145.65,146.60,0.00,0.00%,10,0,"1,665.63%"
4,20.0,AAPL170519C00020000,133.52,135.65,136.60,0.00,0.00%,10,0,"1,210.94%"
5,35.0,AAPL170519C00035000,96.80,96.85,97.60,0.00,0.00%,4,0,0.00%
6,50.0,AAPL170519C00050000,103.10,105.65,106.60,0.00,0.00%,600,0,676.17%
7,70.0,AAPL170519C00070000,82.94,85.65,86.60,0.00,0.00%,222,0,489.84%
8,80.0,AAPL170519C00080000,73.42,75.70,76.60,0.00,0.00%,5,0,419.34%
9,85.0,AAPL170519C00085000,68.45,70.70,71.60,0.00,0.00%,3,0,385.55%


In [2]:
from lxml import objectify

In [8]:
path = os.path.join(data_dir, 'ch06','mta_perf', 'Performance_MNR.xml')

In [9]:
parsed = objectify.parse(open(path))

In [10]:
root = parsed.getroot()

In [12]:
root.INDICATOR

<Element INDICATOR at 0x4e519c8>

In [13]:
data = []

In [14]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [15]:
perf = DataFrame(data)

In [16]:
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [22]:
from io import StringIO

In [23]:
tag = '<a href="http://www.google.com">Google</a>'

In [24]:
root = objectify.parse(StringIO(tag)).getroot()

In [25]:
root

<Element a at 0x4e52a08>

In [26]:
root.get('href')

'http://www.google.com'

In [27]:
root.text

'Google'

In [28]:
ex1_csv = os.path.join(data_dir, 'ch06', 'ex1.csv')

In [29]:
ex1_csv

'E:\\_Workspaces\\JupyterNotebooks\\pydata-book\\ch06\\ex1.csv'

In [30]:
frame = pd.read_csv(ex1_csv)

In [31]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [33]:
frame.to_pickle('frame_pickle')

In [34]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [35]:
store = pd.HDFStore('mydata.h5')

In [37]:
store['obj1'] = frame

In [38]:
store['obj1_col'] = frame['a']

In [39]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
/obj1                frame        (shape->[3,5])
/obj1_col            series       (shape->[3])  

In [40]:
store['obj1']

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [42]:
xls_file = pd.ExcelFile('data.xlsx')

In [43]:
table = xls_file.parse('Sheet1')

In [44]:
table

,item_type,event_type,Event from
0,rule,activation,Rule Activation from ticketing
1,rule,deactivate,NaN
2,rule,disable,Admin disable
3,rule,enable,Admin enable
